In [ ]:
import Logging: global_logger
import TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

In [ ]:
using LinearAlgebra
using SparseArrays
using ModelingToolkit
using MacroTools: @capture, postwalk, prewalk
using DifferentialEquations
using NonlinearSolve
using Plots


Nx = Ny = 75;
harmonics = 1; # number of harmonics
u0_HB = 0.005 * zeros((Nx+1) * (Ny+1) * harmonics * 2);
order = 2;
Nt = 5
N = Nx + 1

xleft::Float64 = 0.0
xright::Float64 = 10.0
yleft::Float64 = 0.0
yright::Float64 = 10.0
stepx = (xright-xleft)/(Nx)
stepy = (yright-yleft)/(Ny)

#Definition of constants (placeholders):
gamma::Float64 = 0.
gamma_3::Float64 = gamma3_final = 0.0
c::Float64 = 0.5
omega::Float64 = 1
A_forcing::Float64 = 250;
lambda_forcing::Float64 = -40;


@parameters x, y, t
@variables u(..)

Dx = Differential(x)
Dy = Differential(y)
Dt = Differential(t)
#Full PDE: Dt(Dt(u(x, y))) - c*c*(Dx(Dx(u(x, y))) + Dy(Dy(u(x, y)))) + gamma*Dt(u(x, y)) + gamma_3*Dt(u(x, y))*Dt(u(x, y))*Dt(u(x, y))
#Reformat as ddu = y_eq
y_eq = c*c*(Dx(Dx(u(x, y))) + Dy(Dy(u(x, y)))) - gamma*Dt(u(x, y)) - gamma_3*Dt(u(x, y))*Dt(u(x, y))*Dt(u(x, y));
# y_eq = expand_derivatives(y_eq)

In [ ]:
function transform_sym(ex)
    return prewalk(ex) do instr
        if @capture(instr, Differential(x)(Differential(x)(s_(x, y))))
            return :(($s[i+1, j]-2* $s[i, j]+$s[i-1, j])/dx^2)
            elseif @capture(instr, Differential(y)(Differential(y)(s_(x, y))))
            return :(($s[i, j+1]-2* $s[i, j]+$s[i, j-1])/dy^2)
            elseif @capture(instr, Differential(x)(s_(x, y)))
            return :(($s[i+1, j] - $s[i-1, j]) / (2*dx))
            elseif @capture(instr, Differential(y)(s_(x, y)))
            return :(($s[i, j+1] - $s[i, j-1]) / (2*dy))
            elseif @capture(instr, Differential(t)(s_(x, y)))
            return :($(Symbol("d$(s)_array"))[i, j])
            elseif @capture(instr, Differential(x, 2)(s_(x, y)))
            return :(($s[i+1, j]-2* $s[i, j]+$s[i-1, j])/dx^2)
            elseif @capture(instr, Differential(y, 2)(s_(x, y)))
            return :(($s[i, j+1]-2* $s[i, j]+$s[i, j-1])/dy^2)
            elseif @capture(instr, Differential(x, 1)(s_(x, y)))
            return :(($s[i+1, j] - $s[i-1, j]) / (2*dx))
            elseif @capture(instr, Differential(y, 1)(s_(x, y)))
            return :(($s[i, j+1] - $s[i, j-1]) / (2*dy))
            elseif @capture(instr, Differential(t, 1)(s_(x, y)))
            return :($(Symbol("d$(s)_array"))[i, j])
            elseif @capture(instr, s_(x, y))
            return :($(Symbol("$(s)_array"))[i, j])
            elseif @capture(instr, s_[i_, j_])
            return :($(Symbol("$(s)_array"))[$(i), $(j)])
        end
        return instr
    end
end
println(y_eq)
println("_____________________________________________")
println(transform_sym(Meta.parse(string(y_eq))))

function create_ODE_function(N, y_expr)
    function_code = quote
        function secODE!(ddu, du, u, p, t)
            dx, dy = p
            grid_size = $N * $N

            u_array = reshape(@view(u[1:grid_size]), $N, $N)
            du_array = reshape(@view(du[1:grid_size]), $N, $N)
            ddu_array = reshape(@view(ddu[1:grid_size]), $N, $N)

            # BCs (Dirichlet for now):
            ddu_array[1,:] .= 0; ddu_array[end,:] .= 0
            ddu_array[:,1] .= 0; ddu_array[:,end] .= 0

            # Inner points:
            for i in 2:$(N-1)
                for j in 2:$(N-1)
                    ddu_array[i, j] = ($y_expr) + A_forcing * exp(lambda_forcing*(i * dx /10.0)^2) * sin(omega * t)
                end
            end

            return ddu
        end
    end
    return eval(function_code)
end

y_expr = transform_sym(Meta.parse(string(y_eq)))

ODEfunc = create_ODE_function(N, y_expr)
u0_FD = zeros(N*N)
du0 = zeros(N*N)
tspan = (0.0, 120.0)
par = [stepx, stepy]
prob = SecondOrderODEProblem(ODEfunc, du0, u0_FD, tspan, par)

@time sol = solve(prob, Tsit5(), saveat=0.1, progress=true, progress_steps=200)

tgrid = sol.t


xgrid = range(xleft, xright, length=N)
ygrid = range(yleft, yright, length=N)


u_data = [reshape(sol.u[k].x[1], N, N) for k in eachindex(sol.t)]

In [ ]:
anim = @animate for (i, t) in enumerate(tgrid)
    heatmap(xgrid, ygrid, u_data[i],
            color=:magma,
            xlabel="x", ylabel="y",
            title="u(x,y,t) at t=$t s",
            clims=(-600, 600),
            aspect_ratio=1)
end

gif(anim, "FD_sol.gif", fps=25)

## Harmonic Balance Solution

In [ ]:
import Symbolics
using DomainSets
include("multiharmonic_balance.jl");
include("jacobian2D.jl");

In [ ]:
function make_WaveEquation_2D(u, g0, height, gamma, gamma3, forcing)
    pde::Symbolics.Num = Dt(Dt(u)) - g0*height*(Dx(Dx(u)) + Dy(Dy(u))) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - forcing;
    return pde
end;

In [ ]:
u0 = rand((Nx + 1) * (Ny + 1) * harmonics * 2)*.4;
solHB = nothing 
u_hist = Vector{Float64}[]  # store history for prediction
gamma3_hist = Float64[]


vars, var_exprs, (u_HB,) = create_ansatz((x, y), t, omega, harmonics);
    
forcing = A_forcing * exp(lambda_forcing*(x/10.0)^2) * sin(omega * t)

bc_conds = Dict(
    :A1 => [:(0.0), 0.0],
    :B1 => [:(0.0), 0.0],
    :A2 => [:(0.0), 0.0],
    :B2 => [:(0.0), 0.0],
    :A3 => [:(0.0), 0.0],
    :B3 => [:(0.0), 0.0],
)

gamma3 = 0.0
dgamma3 = 0.1
resid = nothing

while gamma3 < gamma3_final
    timed_out = false
    println("Beginning the symbolics manipulation")
    flush(stdout)
    
    if length(u_hist) >= 2
        u0 = u_hist[end] + (u_hist[end] - u_hist[end-1]) * 
             dgamma3 / (gamma3_hist[end] - gamma3_hist[end-1])
    elseif length(u_hist) == 1
        u0 = u_hist[end]
    else
        u0 = rand((Nx+1) * (Ny+1) * harmonics * 2)*.3
    end
    
    
    pde::Symbolics.Num = make_WaveEquation_2D(u_HB, c, c, gamma, gamma3, forcing);
    

    expanded = expand_trig_jl(pde, t, omega)
    eqns = make_equations(expanded, harmonics, omega, t)
    sym_eqs = map(transform_sym(Nx, Ny) ∘ Meta.parse ∘ string, eqns)
    
    resid = create_residual_function(sym_eqs, vars, Nx, Ny, bc_conds)
    
    residual! = eval(resid)
    
    DiffMat, LaplCoeff = create_jac_blocks_2D(eqns, var_exprs, harmonics)
    
    jacobian = create_jacobian_function_2D(Nx + 1, DiffMat, LaplCoeff, harmonics)
    
    # Create prototype by calling jacobian! once with dummy sparse matrix
    n = (Nx + 1) *(Ny+1) * harmonics * 2

    jac_prototype = spzeros(n, n)
    jacobian!(jac_prototype, u0, [stepx, stepy])  # fills in the pattern
    
    func = NonlinearFunction(residual!; jac = jacobian!, jac_prototype = jac_prototype)
    prob = NonlinearProblem(func, u0, [stepx, stepy])
    
    solHB = solve(prob, RobustMultiNewton(), reltol=1e-5, abstol=1e-5, maxiters=1000)

    u0 = solHB.u
    println(round(gamma3, sigdigits=6), " ", solHB.retcode)
    if solHB.retcode == ReturnCode.Success
        println("Succeeded!")
        flush(stdout)
        push!(u_hist, copy(solHB.u))
        push!(gamma3_hist, gamma3)
        dgamma3 = min(dgamma3 * 1.2, 0.02)
        
        gamma3 += dgamma3

        if gamma3 > gamma3_final
            gamma3 = gamma3_final
            break
        end
        
    else
        println("Failed! Going back to the previous gamma and reducing the step!")
        flush(stdout)
        gamma3 -= dgamma3
        dgamma3 /= 10
        gamma3 += dgamma3
        dgamma3 < 1e-6 && break
    end


    
end

println("Starting now the FINAL iteration")
flush(stdout)

pde::Symbolics.Num = make_WaveEquation_2D(u_HB, c, c, gamma, gamma3, forcing);
    
expanded = expand_trig_jl(pde, t, omega)
eqns = make_equations(expanded, harmonics, omega, t)
sym_eqs = map(transform_sym(Nx, Ny) ∘ Meta.parse ∘ string, eqns)

resid = create_residual_function(sym_eqs, vars, Nx, Ny, bc_conds);

println("Finished making the residual")
flush(stdout)


residual! = eval(resid)

DiffMat, LaplCoeff = create_jac_blocks_2D(eqns, var_exprs, harmonics)

jacobian = create_jacobian_function_2D(Nx + 1, DiffMat, LaplCoeff, harmonics)

println("Finsihed making the jacobian")
flush(stdout)

# Create prototype by calling jacobian! once with dummy sparse matrix
n = (Nx + 1) *(Ny+1) * harmonics * 2
jac_prototype = spzeros(n, n)
jacobian!(jac_prototype, u0, [stepx, stepy])  # fills in the pattern


println("Defined the problem")
flush(stdout)

func = NonlinearFunction(residual!; jac = jacobian!, jac_prototype = jac_prototype)

# func = NonlinearFunction(residual!)

prob = NonlinearProblem(func, u0, [stepx, stepy])

println("Solving")
flush(stdout)


@time solHB = solve(prob, NewtonRaphson(), reltol=1e-5, abstol=1e-5, maxiters=1000)

print(solHB.retcode)

coefficientsHB = [reshape(solHB.u[(k-1)*(Nx+1)*(Ny+1)+1:k*(Nx+1)*(Ny+1)], Nx+1, Ny+1) for k in 1:(2*harmonics)];

In [ ]:
maxlim  = maximum(coefficientsHB[1])*1.2
x = range(0, 1, length=Nx+1)
y = range(0, 1, length=Ny+1)

T = 2π / omega
n_frames = 500
anim = @animate for t in range(0, T, length=n_frames)
    u_new = coefficientsHB[1] * 0.0
    j = 1
    for i in 1:(2*harmonics)
        if isodd(i)
            u_new .+= coefficientsHB[i] .* sin(j * omega * t)
        else
            u_new .+= coefficientsHB[i] .* cos(j * omega * t)
            j += 1
        end
    end
    
    surface(x, y, u_new', 
        zlims=(-maxlim, maxlim), 
        clims=(-maxlim, maxlim),
        xlabel="x", ylabel="y", zlabel="u",
    title = "H=$harmonics, ω=$(round(omega, digits=1)), " *
            "γ=$gamma, γ₃=$(round(gamma3, sigdigits=2))\n" *
            "Nx=$Nx, Ny=$Ny, t=$(round(t, digits=2)))",
        camera=(30, 30)  # viewing angle
    )
end
gif(anim, "harmonicBalancePlots/WaveEq_gino_$(harmonics)H.gif", fps=60)

## Analysis

In [ ]:
fs_time = fs = 1/0.1
t_steady0 = 500
target = 1 * omega / (2 * pi)

function generateComplexAmplitudeFDSpectrumMatrix(sol, target)
    r1 = zeros(N, N)
    rows = 0
    println(rows)
    for x_idx in 1:1:N
        for y_idx in 1:1:N
            u_at_point = [sol[t][x_idx, y_idx] for t in 1:length(sol)]
            F = fftshift(fft(u_at_point))
            freqs = fftshift(fftfreq(length(u_at_point), fs_time))
            (_, idx) = findmin(abs.(freqs .- target))
            r1[x_idx, y_idx] = abs(F[idx]) / (length(sol) - t_steady0)
        end
    end
    r1
end

using FFTW
fd_complex_spectrum = generateComplexAmplitudeFDSpectrumMatrix(u_data, target)
heatmap(fd_complex_spectrum)

In [ ]:
# coefficientsHB = so/lutions
println(size(coefficientsHB[1]))
function generateComplexAmplitudeHBSpectrumMatrix(h)
    r1 = zeros(N, N)
    rows = 0
    for x_idx in 1:1:N
        for y_idx in 1:1:N
            #f = h * omega / 2π
            r1[x_idx, y_idx] = abs(0.5 * coefficientsHB[2 * h][x_idx, y_idx] - im * 0.5 * coefficientsHB[2 * h - 1][x_idx, y_idx])
        end  
    end
    r1
end
hb_complex_spectrum = generateComplexAmplitudeHBSpectrumMatrix(1)
heatmap(hb_complex_spectrum)